In [ ]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import InceptionV3, ResNet152V2
from tensorflow.keras import Sequential, utils, Input, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization, Dropout, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
import matplotlib.pyplot as plt
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
# Set constant
base_model = ResNet152V2(include_top = False, weights = 'imagenet')
preprocess = tf.keras.applications.resnet_v2.preprocess_input
IMG_SIZE = (512, 512)
BATCH_SIZE = 32

# Build Dataframe

In [ ]:
data_path = "resized_train_cropped/resized_train_cropped/"
print('number of images in total - ',len(os.listdir(data_path)))
data_frame = pd.read_csv("trainLabels_cropped.csv") 
print(data_frame.head(3))
# Getting the additional information from the image name: 
data_frame['patinet_id'] = data_frame['image'].map(lambda x: x.split('_')[0])
#Convert the level into cateorical value: 
data_frame['level_categorical'] = data_frame['level'].map(lambda x: utils.to_categorical(x, 1 + data_frame['level'].max()))

# use pd.concat to join the new columns with your original dataframe
data_frame = pd.concat([data_frame,pd.get_dummies(data_frame['level'], prefix='severity:')],axis=1)
data_frame['data_path'] = data_frame['image'].map(lambda x: os.path.join(data_path, f'{x}.jpeg'))
print(data_frame.sample(3))
data_frame['level'].hist()

# Build Data Generator

In [ ]:
# 5:1 train:validation
train, validation = train_test_split(data_frame, test_size=0.2)

train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=5,
    height_shift_range=5,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True, 
    preprocessing_function=preprocess
    )
test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess)

def get_data_set(dataframe, data_generator):
    return data_generator.flow_from_dataframe(
        dataframe,
        directory=None,
        x_col="data_path",
        y_col='level',
        weight_col=None,
        target_size=IMG_SIZE,
        color_mode="rgb",
        classes=None,
        class_mode="raw",
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=None,
        save_to_dir=None,
        save_prefix="",
        save_format="png",
        subset=None,
        interpolation="nearest",
        validate_filenames=False,
    )
train_set = get_data_set(train, train_data_gen)
val_set = get_data_set(validation, test_data_gen)

# Build Model

In [ ]:
base_model.trainable = False
    
image_shape = next(val_set)[0].shape[1:]
x0 = Input(image_shape)
x = base_model(x0)
head = Sequential([
    Flatten(),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer='l2'),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer='l2'),
    Dense(5, activation='softmax', kernel_regularizer='l2')
])
y = head(x)
model = Model(inputs=x0, outputs=y)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
model.summary()



# Train Model

In [ ]:
filepath = base_model.name + "-epoch{epoch:02d}-{val_sparse_categorical_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, 
                             save_best_only=False, mode='auto', save_weights_only=False, save_freq='epoch')

tensorboard = TensorBoard(log_dir="logs", histogram_freq=0, write_graph=True, write_images=True,
                               update_freq="epoch", profile_batch=2, embeddings_freq=0, embeddings_metadata=None)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3,
                                   verbose=1, mode='auto', min_delta=0.0001, cooldown=5, min_lr=0.0001)
earlystop = EarlyStopping(monitor="val_loss", mode="auto", patience=6)

callbacks_list = [checkpoint, tensorboard, reduceLROnPlat, earlystop]

fitting = model.fit(
    train_set, 
    validation_data = val_set, 
    epochs = 10,
    verbose = 1,
    callbacks = callbacks_list
)
# Plot fitting the val_sparse_categorical_accuracy and loss of each epoch
print(fitting.history.keys())
plt.plot(fitting.history['val_sparse_categorical_accuracy'])
plt.title('val_sparse_categorical_accuracy')
plt.figure()
plt.plot(fitting.history['loss'])
plt.title('loss')